In [2]:
import os, sys
from labelbox import Client, DataRow
from uuid import uuid4 ## to generate unique IDs
import datetime 
sys.path.insert(0, os.path.abspath('..'))
import config

import boto3
from labelbox import Client




2023-07-08 20:20:53:rastervision.pipeline.rv_config: WARNING - Root temporary directory cannot be used: /data/tmp. Using root: /data/tmp/tmpjawszlh1


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
%reload_ext autoreload
import utils.s3_utils as s3_utils

# Fetch all API KEYs
Make sure your labelbox api key is securely stored in your environment variables. You can find your api key in your labelbox account settings.

Similarly make sure you have the AWS Public and secret keys stored in your environment variables. You can find your AWS keys in your AWS account settings.

In [5]:
lb_apikey = os.environ.get('LABELBOX_API_KEY')
aws_pubkey = os.environ.get('AWS_ACCESS_KEY')
aws_secret = os.environ.get('AWS_SECRET_KEY')
s3_bucket = 'sandmines'
s3_folder = 'test/'
s3_location = 'us-west-1'

## Setup S3

In [11]:
s3 = boto3.client('s3',
                    aws_access_key_id=aws_pubkey, 
                    aws_secret_access_key=aws_secret, 
                    region_name=s3_location)

In [12]:
folders = s3_utils.list_folders(s3, 'sandmines')
print (folders)

['labels/', 'test/']


## Set observation name

In [13]:
dataset_name = 'Sone_Dehri_XX-xx_YY-yy'

## Fetch link to timelapse

In [14]:
key = s3_folder + dataset_name 
files = s3_utils.list_files(s3, s3_bucket, key)

#filter for the files with '_timelapse' in the name
timelapse_file = [file for file in files if 'timelapse' in file][0]

timelapse_url = s3_utils.get_public_url(s3_location, s3_bucket, timelapse_file)
print (timelapse_url)

https://s3-us-west-1.amazonaws.com/sandmines/test/Sone_Dehri_XX-xx_YY-yy/Sone_Dehri_XX.xx_YY.yy_timelapse.mp4


## Fetch all RGB files in the dataset

In [15]:
key = s3_folder + dataset_name + '/rgb'
files = s3_utils.list_files(s3, s3_bucket, key)

print (files)

['test/Sone_Dehri_XX-xx_YY-yy/rgb/Sone_Dehri_XX-xx_YY-yy_YYYYMM22_rgb.tif', 'test/Sone_Dehri_XX-xx_YY-yy/rgb/Sone_Dehri_XX-xx_YY-yy_YYYYMMDD_rgb.tif']


In [16]:
# for each item in files, create a new dictionary with the following keys:
#"row_data" : the url of the image, get this using the get_public_url function
#"global_key" : the file name, get this from the file variable, but remove the folder name and the .tif extension

data_rows = []
global_keys = []

for file in files:
    rgb_url = s3_utils.get_public_url(s3_location, s3_bucket, file)
    global_key = file.split('/')[-1].split('.')[0]
    # print (global_key, url)

    this_row_data = {
        "tile_layer_url": rgb_url,
        "min_zoom": 12,
        "max_zoom": 20,
        "epsg": "EPSG4326",
        "name" : "RGB",
        "alternative_layers": [
          {
            "tile_layer_url": "https://api.mapbox.com/styles/v1/mapbox/satellite-v9/tiles/{z}/{x}/{y}?access_token=pk.eyJ1IjoiYW5kby1zaGFoIiwiYSI6ImNsanFmNDJiYzA1ZHYzaW5nazY4YWU3cDEifQ.mABfTStcXV1cMQyAna8_gQ",
            "name": "Hi-res Guidance"
          }
        ]
    }

    data_rows.append({'row_data' : this_row_data,
                      'global_key' : global_key,
                      "media_type": "TMS_GEO",
                        "attachments": [{
                            "type": "VIDEO",
                            "value": timelapse_url
                            }]
                    }) 
    global_keys.append(global_key)
data_rows

[{'row_data': {'tile_layer_url': 'https://s3-us-west-1.amazonaws.com/sandmines/test/Sone_Dehri_XX-xx_YY-yy/rgb/Sone_Dehri_XX-xx_YY-yy_YYYYMM22_rgb.tif',
   'min_zoom': 12,
   'max_zoom': 20,
   'epsg': 'EPSG4326',
   'name': 'RGB',
   'alternative_layers': [{'tile_layer_url': 'https://api.mapbox.com/styles/v1/mapbox/satellite-v9/tiles/{z}/{x}/{y}?access_token=pk.eyJ1IjoiYW5kby1zaGFoIiwiYSI6ImNsanFmNDJiYzA1ZHYzaW5nazY4YWU3cDEifQ.mABfTStcXV1cMQyAna8_gQ',
     'name': 'Hi-res Guidance'}]},
  'global_key': 'Sone_Dehri_XX-xx_YY-yy_YYYYMM22_rgb',
  'media_type': 'TMS_GEO',
  'attachments': [{'type': 'VIDEO',
    'value': 'https://s3-us-west-1.amazonaws.com/sandmines/test/Sone_Dehri_XX-xx_YY-yy/Sone_Dehri_XX.xx_YY.yy_timelapse.mp4'}]},
 {'row_data': {'tile_layer_url': 'https://s3-us-west-1.amazonaws.com/sandmines/test/Sone_Dehri_XX-xx_YY-yy/rgb/Sone_Dehri_XX-xx_YY-yy_YYYYMMDD_rgb.tif',
   'min_zoom': 12,
   'max_zoom': 20,
   'epsg': 'EPSG4326',
   'name': 'RGB',
   'alternative_layers': [{'t

## Setup LabelBox client

In [19]:
client = Client(api_key=lb_apikey)

In [ ]:
dataset = client.create_dataset(name=dataset_name)

## Send all images to LabelBox

In [72]:
task = dataset.create_data_rows(data_rows)
task.wait_till_done()
print(task.errors)

None


# Download labeled data rows from LabelBox

Only run this AFTER the labeling step is complete in the LabelBox cloud UI.
DO NOT run it imediately after sending the images to LabelBox. It will return an empty dataset.


`CURRENTLY NOT WORKING`

In [20]:
# this tests that the global keys were able to fetch all of the data rows we expect
res = client.get_data_row_ids_for_global_keys(global_keys)

print(res)
# ---


{'status': 'SUCCESS', 'results': ['cljujcxmg0acs075k9sjo45gz', 'cljujcxmg0acz075k96vf2tuz'], 'errors': []}


In [21]:
project_id = 'cljoqjqbu065d070wga4ue4jl'  #from LabelBox > Schema > <Project Name> General > ID
project = client.get_project(project_id)

ResourceNotFoundError: Resource 'Project' not found for params: {'param_0': 'cljoqjqbu065d070wga4ue4jl'}("Resource 'Project' not found for params: {'param_0': 'cljoqjqbu065d070wga4ue4jl'}", None)

In [ ]:
export_params= {
  "attachments": False,
  "metadata_fields": False,
  "data_row_details": False,
  "project_details": False,
  "label_details": False,
  "performance_details": False,
  "interpolated_frames": False
}

# Note: Filters follow AND logic, so typically using one filter is sufficient.
filters= {
  "data_row_ids": global_keys 
}

export_task = project.export_v2(params=export_params, filters=filters)
export_task.wait_till_done()

if export_task.errors:
  print(export_task.errors)

export_json = export_task.result
print("results: ", export_json)